#### Objective : This code make use of complaint data against financial companies to perform following tasks on the data
    1. Create Topic models to classify comlplaints in various categories.
    2. We will do test preprcessingw here which remove all token which belong to org and other preprocessing
    3. We will try out two Topic Modelling Algorithms
        1. Latent Dirichtlet ALogorithm - In my exp does not work well on short text data
        2. Non Negative Matrix factorisation

#### About the Dataset
The dataset comprises of Consumer Complaints on Financial products and we’ll see how to classify consumer complaints text into these categories: Debt collection, Consumer Loan, Mortgage, Credit card, Credit reporting, Student loan, Bank account or service, Payday loan, Money transfers, Other financial service, Prepaid card.
Also we will try to identify the companies from the dataset

#### Import the required package

In [ ]:
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import nltk
import spacy
import seaborn as sns
import pickle
from nltk.stem import PorterStemmer
import string

### Read the complaints data csv

In [ ]:
complaint_data = pd.read_csv("../input/consumer-complaints-financial-products/Consumer_Complaints.csv",low_memory = False)

In [ ]:
### Convert the columns names so that they don't have space and are more readable
complaint_data.columns = [i.lower().replace(" ","_").replace("-","_") for i in complaint_data.columns]
complaint_data.columns

### Lets understand the shape an dtypes of the data

In [ ]:
print ("The shape of data is ",complaint_data.shape)
print ("The data types for our data are as follows ")


In [ ]:
### All the varables are text - which may correspond to categories and other variables
print (" The number of unique values in each column is as follows")
### Lets do a describe with including objects
complaint_data.describe(include = 'object').T.reset_index()


######  From above description we see that only 114704 rows have complaint text and as we are interested in only those row which have complaint text. We wll drop all rows where complaint narrative is na

In [ ]:
#### Keep only the consumer complaints is not null
complaint_data = complaint_data[~complaint_data['consumer_complaint_narrative'].isna()]

In [ ]:
#### Create a distirbution of length of customers complaints. We have very left skew in length of complaints
### Which is expected as most compalints can be written in less than 500 words
complaint_data['consumer_complaint_narrative'].apply(len).plot(kind = 'hist')

In [ ]:
### Keep the length columns as a new column
complaint_data['comp_length'] = complaint_data['consumer_complaint_narrative'].apply(len)
complaint_data.reset_index(inplace = True)
complaint_data['consumer_complaint_narrative'] = complaint_data['consumer_complaint_narrative'].str.replace(r"XX+\s","")
complaint_data['consumer_complaint_narrative'] = complaint_data['consumer_complaint_narrative'].str.replace("XXXX","")

In [ ]:
complaint_data = complaint_data.sample(frac = 0.5,random_state = 5)
complaint_data.reset_index(inplace = True)

### We will first craete the data processing pipeline to do cleaning on the data

In [ ]:
nlp = spacy.load('en_core_web_sm')
### Remove the stop words using spacy predefined list 
stop_words = nlp.Defaults.stop_words
#### Create a list of puntuation to be removed
symbols = " ".join(string.punctuation).split(" ") 
### As we are doing topic modelling itsa good idea to do lemmatisation - as it uses morphologial analysis
ps = PorterStemmer()
import re
#### Lets define the cleaning function and see how it works
def cleanup_text(docs,logging = False):
    texts = []
    counter = 1
    for doc in docs:
        
        if counter % 5000 == 0 :
            print ("Processed %d of out of %d documents"% (counter,len(docs)))
        counter += 1
        
        doc = nlp(doc) ### We are disabling parser as will nt be using it
        
        
        tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != "-PRON-"]
        
        tokens =[tok for tok in tokens if tok not in symbols]
        tokens = [tok for tok in tokens if tok not in stop_words]
        tokens = [re.sub('[0-9]', '', i) for i in tokens]
        tokens = ' '.join(tokens)
        texts.append(tokens)
    return (pd.Series(texts))

In [ ]:
complaint_data['comp_preprocessed'] = cleanup_text(complaint_data['consumer_complaint_narrative'])

In [ ]:
print ("Shape of data before removing NA's ,",complaint_data.shape)
complaint_data =complaint_data[~complaint_data['comp_preprocessed'].isna()]
print ("Shape of data before removing NA's ,",complaint_data.shape)

In [ ]:
complaint_data[['comp_preprocessed','consumer_complaint_narrative']].head(5)

### Lets tranform the data using TFIDF vectoriser
1. For learning purpose i am using only tf-idf , but inpractical it as advised to trial out count vectoriser also
2. Also, all paarmeters in TF IDF can be treated as Hyperparameters

In [ ]:
### Lets Create the piprle line for NMF models 
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF 
#####  Let extract from act the features from the dataset

print ("Extracting the tf-idf features form NMF")
tfidf_vectorizer = TfidfVectorizer(max_df = 0.5, min_df = 5, max_features = 500, ngram_range = (1,4))

t0 = time()
tfidf = tfidf_vectorizer.fit_transform(complaint_data['comp_preprocessed'])
print ("done in %0.3fs." % (time() - t0))

In [ ]:
#### Now we will fit model for 10 diff values of clusters
n_comp = [10,20,30,40,50,60,70,80,90,100,110]
loss = []
for comps in n_comp:
  
    t0 = time()
    nmf = NMF(n_components = comps, random_state = 1, beta_loss = 'kullback-leibler',solver = 'mu',max_iter = 200,
             alpha = 0.1, l1_ratio = 0.5).fit(tfidf)
    loss.append(nmf.reconstruction_err_)
    print ("done in %0.3f " % (time() -t0))

In [ ]:
### Let try to create a elbow and find out the best model clusters
plt.plot(loss)
plt.xlabel('Number of Topics')
plt.ylabel('Reconstruction Error - Frobenius Norm')

### Define a function which goes through - topic * word matrix and extract the top keywords

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [ ]:
print_top_words(nmf, tfidf_vectorizer.get_feature_names(), 10)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
# Fit the NMF model
#### Now we will fit model for 10 diff values of clusters
n_comp = [10,20,30,40,50]

for comps in n_comp:
    loss1 = []
    t0 = time()
    lda = LatentDirichletAllocation(n_components=comps, max_iter=2,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
    lda.fit(tfidf)
    print("done in %0.3fs." % (time() - t0))

In [ ]:
print_top_words(lda, tfidf_vectorizer.get_feature_names(), 10)

### Next Steps
1. Data needs a lot of cleaning for example "XX","XXX",organisation names
2. After looking at Topics we realise that a lot of keywords are important but do not give anyinformation about complaints. We should create a custom stop word list & clean the required data
3. We may want to try clustering topics into K means for naming and better understanding and grouping
4. Use LDA visualisation to understand topic in much more details